In [7]:
import pandas as pd
from chat_bots import ChatSentimento, ChatFundamentalistas, ChatAnaliseTecnica, ChatBot, ChatValuation, ChatTradutor
from coleta_dados import (DadosFundamentalistas, 
                          DadosNoticiasBuscadorYahoo, 
                          DadosNoticiasGoogle, 
                          CalculoWACC, 
                          VariacaoReceita, 
                          ValuationModoloGordon, 
                          OutrosAtivosNaoOperacionais,
                          PassivoTotalMenosDivida,
                          NecessidadeCapitalGiro,
                          IndicadoresFinanceiros,
                          ValuationFluxoCaixaDescontado)

from tratando_dados import tratando_dados_fundamentalistas, TratarDadosNoticias, TratandoDadosIndicadores, TratandoDadosValuation
from juncao_modelos_dados import ModeloValuation, ModeloFundamentos, ModeloSentimento, ModeloAnaliseTecnica
from dotenv import load_dotenv
import ipeadatapy as ip
from datetime import datetime
import pandas as pd
import yfinance as yf
import sidrapy
import warnings
warnings.filterwarnings('ignore')
load_dotenv()


True

# Fundamentalista

In [3]:
tic = 'BBDC3'
data_inicio = '2020-01-01'

dados_fundamentalistas = DadosFundamentalistas(tic, data_inicio)
trat = tratando_dados_fundamentalistas(tic, data_inicio)

In [6]:
from typing import List, Any
def retransfromando_pandas(trat: List[Any]) -> pd.DataFrame:
    df = pd.DataFrame([doc.metadata for doc in trat])

    # Adicionar a coluna "data" com base no page_content
    df['data'] = [doc.page_content for doc in trat]

    return df

In [7]:

df = retransfromando_pandas(trat)

df

,receita_liquida,resultado_bruto,lucro_liquido,tic,capex_tres_meses,fluxo_caixa_operacional,fluxo_caixa_investimentos,fluxo_caixa_financiamento,aumento_reducao_caixa_equivalentes,preço_lucro,...,patrimonio_liquido,valor_patrimonial_acao,retorno_sobre_patrimonio_liquido_inicial,retorno_sobre_ativo_inicial,margem_bruta,margem_liquida,giro_do_ativo_inicial,alavancagem_financeira,passivo_patrimonio_liquido,data
0,56450.0,19900.0,4150.00,BBDC3,-300.42,10250.0,-9710.0,-5280.0,-4850.0,6.711.15,...,168410.0,0.02,0.104,0.009,0.285,0.071,0.13,12.29,11.29,2024-12-31
1,52500.0,15210.0,4870.00,BBDC3,-10.25,-6280.0,8500.0,-4050.0,-1930.0,9.32,...,167130.0,15.70,0.088,0.008,0.277,0.061,0.13,12.07,11.07,2024-09-30
2,51500.0,16940.0,4120.00,BBDC3,-10.25,28570.0,-6580.0,-13280.0,8790.0,9.96,...,164630.0,15.52,0.078,0.007,0.277,0.054,0.13,12.17,11.17,2024-06-30
3,52770.0,16890.0,4120.00,BBDC3,-205.93,17680.0,2780.0,-1060.0,19220.0,10.16,...,165860.0,15.59,0.080,0.007,0.272,0.053,0.13,11.79,10.79,2024-03-31
4,54110.0,17050.0,1440.00,BBDC3,-393.47,-81010.0,7160.0,8760.0,-65010.0,9.43,...,166330.0,15.63,0.090,0.008,0.271,0.057,0.14,11.59,10.59,2023-12-31
5,56070.0,15920.0,3400.00,BBDC3,-138.21,79570.0,9870.0,-10620.0,78880.0,10.91,...,166230.0,15.62,0.082,0.007,0.271,0.052,0.14,11.40,10.40,2023-09-30
6,56850.0,17140.0,4020.00,BBDC3,6.56,-19740.0,15050.0,-10250.0,-15030.0,10.34,...,166450.0,15.64,0.096,0.009,0.275,0.061,0.14,11.19,10.19,2023-06-30
7,57430.0,17980.0,5400.00,BBDC3,-77.89,21000.0,51520.0,-10960.0,61760.0,7.21,...,162490.0,15.24,0.119,0.011,0.294,0.076,0.14,11.25,10.25,2023-03-31
8,53640.0,18270.0,785.85,BBDC3,-178.48,-32520.0,3860.0,-1860.0,-30350.0,6.27,...,158270.0,14.85,0.140,0.012,0.320,0.090,0.14,11.37,10.37,2022-12-31
9,53140.0,17100.0,5840.00,BBDC3,-368.35,43650.0,-10720.0,13450.0,46420.0,6.44,...,161360.0,15.14,NaN,NaN,0.340,0.119,NaN,11.54,10.54,2022-09-30


In [6]:
query = 'Como esta a saude financeira da bbdc3'
response = ChatFundamentalistas(query=query, dados=trat,modelo_llm="deepseek-r1-distill-llama-70b")

In [7]:
response.split('\n\n')

['<think>\nOk, preciso analisar a saúde financeira da BBDC3 com base nos indicadores fornecidos. Vou começar identificando as informações disponíveis e entendendo o que cada uma delas significa.',
 'Primeiro, vou olhar para a receita líquida. Vejo que ela tem flutuado um pouco, mas parece estar em uma tendência de crescimento gradual. Isso é positivo porque indica que a empresa está aumentando sua receita ao longo do tempo.',
 'Em seguida, o lucro líquido. Notando que o lucro líquido também tem variações, mas em alguns períodos foi bastante alto, como 7.870,00, mas em outros foi muito baixo, como 1.440,00. Isso pode indicar volatilidade nos resultados, o que pode ser preocupante.',
 'Margens brutas e líquidas estão estáveis, o que é bom. A margem bruta em torno de 27% e a líquida em torno de 5-6% indicam que a empresa mantém uma rentabilidade consistente, embora possa haver espaço para melhoria na margem líquida.',
 'Agora, olhando para os fluxos de caixa. O fluxo operacional tem sido 

In [2]:
query = 'Como esta a saude financeira da PETR4'
tic = 'PETR4'
fund = ModeloFundamentos(query=query, ticker=tic)

response, dados = fund.chat_fundamentalistas()


In [3]:
response

<generator object RunnableSequence.stream at 0x0000027327FF65C0>

In [7]:
retransfromando_pandas(dados)


,receita_liquida,resultado_bruto,ebit,amortizacao,ebitda,lucro_liquido,lucro_por_acao,tic,capex_tres_meses,fluxo_caixa_livre_tres_meses,...,retorno_sobre_capital_investido_inicial,retorno_sobre_patrimonio_liquido_inicial,retorno_sobre_ativo_inicial,margem_bruta,margem_liquida,margem_ebit,giro_do_ativo_inicial,alavancagem_financeira,passivo_patrimonio_liquido,data
0,129580.0,66580.0,46460.0,-16540.0,63000.0,32560.0,2.50,PETR4,-24640.0,38080.0,...,0.171,0.219,0.083,0.518,0.168,0.325,0.49,2.74,1.74,2024-09-30
1,122260.0,61050.0,33540.0,-16360.0,49900.0,-2610.0,- 0.20,PETR4,-15260.0,31910.0,...,0.181,0.212,0.080,0.522,0.158,0.330,0.50,2.83,1.83,2024-06-30
2,117720.0,60700.0,44030.0,-16650.0,60680.0,23700.0,1.82,PETR4,-14050.0,32430.0,...,0.184,0.274,0.113,0.525,0.224,0.353,0.50,2.62,1.62,2024-03-31
3,134260.0,72630.0,39630.0,-20490.0,60120.0,31040.0,2.38,PETR4,-17790.0,39860.0,...,0.208,0.344,0.128,0.527,0.243,0.370,0.52,2.76,1.76,2023-12-31
4,124830.0,66320.0,47550.0,-19480.0,67030.0,26630.0,2.04,PETR4,-15540.0,40990.0,...,0.213,0.368,0.144,0.511,0.255,0.387,0.57,2.66,1.66,2023-09-30
5,113840.0,57680.0,41970.0,-18280.0,60250.0,28780.0,2.21,PETR4,-14390.0,33360.0,...,0.256,0.381,0.156,0.506,0.269,0.403,0.58,2.67,1.67,2023-06-30
6,139070.0,73310.0,60200.0,-17770.0,77970.0,38160.0,2.93,PETR4,-12590.0,41170.0,...,0.303,0.419,0.182,0.521,0.285,0.453,0.64,2.43,1.43,2023-03-31
7,158580.0,76640.0,57810.0,-19460.0,77270.0,43340.0,3.32,PETR4,-18710.0,48870.0,...,0.296,0.486,0.194,0.521,0.294,0.459,0.66,2.69,1.69,2022-12-31
8,170080.0,86840.0,74610.0,-18540.0,93150.0,46100.0,3.53,PETR4,-10230.0,52980.0,...,0.307,0.478,0.181,0.513,0.286,0.478,0.63,2.54,1.54,2022-09-30
9,170960.0,95860.0,96430.0,-18620.0,115050.0,54330.0,4.16,PETR4,-8380.0,63420.0,...,0.302,0.454,0.172,0.509,0.284,0.504,0.60,2.45,1.45,2022-06-30


In [2]:
query = 'Como esta a saude financeira da bbdc3'
tic = 'BBDC3'
fund = ModeloFundamentos(query=query, ticker=tic)

In [42]:
response = fund.chat_fundamentalistas()

In [5]:
for chunk in response: 
    print(chunk, end="", flush=True)

<think>
Ok, vou analisar a saúde financeira da BBDC3 com base nos dados fornecidos. Primeiro, vou olhar para a receita líquida e ver se ela está crescendo ou não. Vejo que a receita líquida tem flutuado nos últimos períodos, com um pico em 2021-12-31 e uma queda recente. Isso pode indicar instabilidade.

Em seguida, vou verificar o lucro líquido. Observo que em 2024-09-30 o lucro líquido caiu para R$ 4.870, o que é preocupante. A margem líquida também diminuiu, saindo de 11,9% para 6,1%. Isso sugere que a empresa está enfrentando desafios para manter sua rentabilidade.

Agora, vou analisar os fluxos de caixa. O fluxo operacional foi negativo em 2024-09-30, o que é um sinal de alerta. No entanto, em outros períodos ele foi positivo, então preciso considerar se isso é um problema contínuo ou um evento isolado. O fluxo de investimentos tem sido volátil, com altos e baixos, o que pode indicar investimentos irregulares.

O fluxo de financiamento também tem variado bastante, com saídas signi

# Sentimento

In [8]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-features=NetworkService")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-features=VizDisplayCompositor")

dados_yahoo = DadosNoticiasBuscadorYahoo(acao='BBDC3', options=chrome_options)


In [3]:
dados_ya = dados_yahoo.get_news(number_paginas=1)

Sem mais páginas!


In [12]:
dados_ya

{'links': ['https://www.infomoney.com.br/onde-investir/agenda-de-dividendos-de-fevereiro-2025/',
  'https://einvestidor.estadao.com.br/mercado/ibovespa-hoje-magazine-luiza-mglu3-salta-so-4-acoes-fecham-queda/',
  'https://einvestidor.estadao.com.br/ultimas/ibovespa-hoje-magazine-luiza-mglu3-dispara-9-braskem-brkm5-sofre/',
  'https://einvestidor.estadao.com.br/investimentos/dividendos-da-semana-bradesco-taesa-e-mais-6-empresas-pagam-acionistas-2/',
  'https://www.infomoney.com.br/mercados/7-acoes-sobem-mais-de-20-e-10-caem-mais-de-5-os-destaques-do-ibovespa-em-janeiro/'],
 'titulos': ['Agenda de dividendos de fevereiro: Petrobras, Itaú e Santander pagam neste mês',
  'Ibovespa hoje: Magazine Luiza (MGLU3) salta mais de 12%; só 4 ações fecham em queda',
  'Ibovespa hoje: Magazine Luiza (MGLU3) dispara 10% e lidera ganhos; Braskem (BRKM5) sofre',
  'Dividendos da semana: Bradesco (BBDC4), Banestes (BEES4), Itaú (ITUB4) e mais 9 empresas pagam...',
  '7 ações sobem mais de 20% e 10 caem m

In [9]:
ticker = 'BBDC3'
from dotenv import load_dotenv
load_dotenv()
import os

api_secret_groq = os.getenv("GROQ_API_KEY")
api_secret_serper = os.getenv("SERPER_API_KEY")

dados_google = TratarDadosNoticias(ticker, chrome_options, api_secret_groq, api_secret_serper)

In [10]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-features=NetworkService")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-features=VizDisplayCompositor")

ticker = 'BBDC3'
news_tratado = TratarDadosNoticias(ticker, chrome_options, api_secret_groq, api_secret_serper)
dados_new = news_tratado.clean_chat_html_bs4()

Error fetching URL: Client error '403 Forbidden' for url 'https://riconnect.rico.com.vc/analises/dividendos-de-bradesco-bbdc4-bbdc3-quem-recebe-e-quando/'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Error fetching URL: Redirect response '302 Found' for url 'https://br.advfn.com/noticias/PAPERBR/2025/artigo/95421182'
Redirect location: '/jornal/2025/02/goldman-sachs-rebaixa-recomendacao-da-acao-do-bradesco-devido-a-expectativa-de-recuperacao-lenta-da-lucratividade'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/302
Error fetching URL: Client error '403 Forbidden' for url 'https://seucreditodigital.com.br/bradesco-2025-oportunidades-desafios/'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403


In [9]:
query = f'Qual é o sentimento sobre as notícias sobre a {ticker}'
dados_noticia = ChatSentimento(query=query, noticia=dados_new, modelo_llm="deepseek-r1-distill-llama-70b")

In [10]:
dados_noticia.split('\n\n')

['<think>\nOkay, I need to evaluate the overall sentiment of recent news about BBDC3 for an investment analysis. Let me start by going through each news article provided.',
 'First, I see several mentions of dividend payments and JCP (juros sobre capital próprio). These are positive indicators because they show that the company is generating enough profit to distribute to shareholders. High dividends can attract investors and indicate financial health.',
 "Next, there are articles about the company's strong financial results. For example, the lucro líquido (net income) was above expectations, with a 12% increase. This is a strong positive signal as it reflects good performance and stability.",
 'The expansion into international markets is another positive point. Entering new markets can lead to growth and diversification, which is great for long-term investment.',
 "The launch of a stock option plan is also positive. This can increase liquidity and attract more investors, which is bene

In [13]:
ticker = 'BBDC3'
query = f'Qual é o sentimento sobre as notícias sobre a {ticker}'
sent = ModeloSentimento(query=query, acao=ticker)

response = sent.chat_sentimento()

Error fetching URL: Client error '403 Forbidden' for url 'https://riconnect.rico.com.vc/analises/dividendos-de-bradesco-bbdc4-bbdc3-quem-recebe-e-quando/'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Error fetching URL: Redirect response '302 Found' for url 'https://br.advfn.com/noticias/PAPERBR/2025/artigo/95421182'
Redirect location: '/jornal/2025/02/goldman-sachs-rebaixa-recomendacao-da-acao-do-bradesco-devido-a-expectativa-de-recuperacao-lenta-da-lucratividade'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/302
Error fetching URL: Client error '403 Forbidden' for url 'https://seucreditodigital.com.br/bradesco-2025-oportunidades-desafios/'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403


In [14]:
for chunk in response: 
    print(chunk, end="", flush=True)

<generator object RunnableSequence.stream at 0x00000252960B3880>
New notice
ra Bradesco (BBDC3; BBDC4): comprar ou vender?
Outros papéis de banco também caem em bloco no pregão
As ações do Bradesco fecharam em queda no pregão desta quarta-feira (12). Os papéis ordinários (BBDC3) encerraram em recuo de 3,71%, cotados a R$ 10,64. Já os preferenciais (BBDC4) despencaram mais: 4,56%, negociados a R$ 11,72.
Nesta quarta, o Goldman Sachs cortou a recomendação da ação de compra para venda, com preço-alvo de R$ 11,40, devido a revisões de estimativas. A instituição afirma que o desempenho abaixo da média da ação está relacionado principalmente à recuperação lenta da rentabilidade, que ainda permanece abaixo dos concorrentes.
Mercado espera agora apenas um corte de juros nos EUA em 2025; veja quando
Tesouro Direto: taxas so
New notice
eiro
Bradesco (BBDC3;BBDC4) pagará JCP mensais em 2025; veja datas e valores
Banco é uma das poucas empresas brasileiras que distribuem proventos mensalmente aos 

# Analise tecnica

In [53]:
ticker = 'BBDC3'
trat_indi = TratandoDadosIndicadores(ticker=ticker, periodo= "18Y", intervalo="1mo" )


In [54]:
dados_indicadores =   trat_indi.indicadores_data_loader()
dados_indicadores

[Document(metadata={'20 period SMA': 12.107581233978271, '100 period SMA': 12.548377685546875, '200 period SMA': 9.040361684560775, '14 period RSI': 44.05830300243827, 'MACD': -0.49224890697876056, 'SIGNAL': -0.5226269899657147, 'BB_UPPER': 14.534689850428775, 'BB_MIDDLE': 12.107581233978271, 'BB_LOWER': 9.680472617527768, 'pivot': 11.438830689693523, 's1': 10.849385519039558, 's2': 10.334743061089366, 's3': 9.7452978904354, 's4': 9.155852719781436, 'r1': 11.953473147643715, 'r2': 12.54291831829768, 'r3': 13.057560776247872, 'r4': 13.572203234198064, 'VWAP.': 10.689990138811522, '14 period ADX.': 13.103405767068136, 0: 10.104291757718583}, page_content='2023-10-01'),
 Document(metadata={'20 period SMA': 12.066738319396972, '100 period SMA': 12.596793308258057, '200 period SMA': 9.084436701536179, '14 period RSI': 52.005257604367536, 'MACD': -0.3734190004541098, 'SIGNAL': -0.49278539206339367, 'BB_UPPER': 14.40918682369609, 'BB_MIDDLE': 12.066738319396972, 'BB_LOWER': 9.724289815097855,

In [6]:
response = ChatAnaliseTecnica(
    query=f"Qual é a tendencia sobre a empresa {ticker}",
    dados=dados_indicadores,
    modelo_llm="deepseek-r1-distill-llama-70b",
    stream=True
)

In [7]:
for chunk in response:
    print(chunk, end="", flush=True)

<think>
Okay, I need to analyze the financial data for BBDC3 to determine the current trend and recommend a trading strategy. Let me go through each indicator step by step.

First, looking at the Moving Averages. The 20-period SMA is around 11.58, the 100-period is about 12.59, and the 200-period is 9.33. The price is above the 200-period SMA, which is a good sign, but it's below the 100-period SMA. That might indicate a short-term downtrend within a longer-term uptrend.

Next, the RSI is at 46.38. Since it's between 30 and 70, the asset isn't oversold or overbought. That means there's no clear signal from the RSI right now.

Now, the MACD is negative (-0.335) and the Signal line is also negative (-0.281). The MACD is below the Signal line, which is a bearish sign. This could mean a sell signal or that the downtrend might continue.

Looking at Bollinger Bands, the Middle Band is at 11.59, and the price is slightly below it. The Bands are contracting, which suggests low volatility. This

In [8]:
ticker = 'BBDC3'
query = f"Qual é a tendencia sobre a empresa {ticker}"
tec = ModeloAnaliseTecnica(query=query,ticker=ticker)

In [ ]:
response, dados_tecnicas = tec.chat_analise_tecnica()
for chunk in response: 
    print(chunk, end="", flush=True)

In [11]:
retransfromando_pandas(dados_tecnicas)

,20 period SMA,100 period SMA,200 period SMA,14 period RSI,MACD,SIGNAL,BB_UPPER,BB_MIDDLE,BB_LOWER,pivot,...,s3,s4,r1,r2,r3,r4,VWAP.,14 period ADX.,0,data
0,11.894986,12.328042,8.881623,44.058308,-0.483605,-0.513450,14.279478,11.894986,9.510494,11.237979,...,9.574183,8.995087,11.743585,12.322680,12.828286,13.333892,10.502286,13.103393,9.926872,2023-10-01
1,11.854861,12.375608,8.924924,52.005260,-0.366862,-0.484132,14.156179,11.854861,9.553542,11.011312,...,10.004664,9.616134,11.240898,11.629429,11.859015,12.088601,10.516693,12.943196,10.064023,2023-11-01
2,11.927965,12.439244,8.971850,55.471372,-0.199183,-0.427142,14.347634,11.927965,9.508297,12.125994,...,9.299346,8.524155,13.402259,14.177450,15.453715,16.729980,10.537828,13.574619,10.195688,2023-12-01
3,11.867936,12.496842,9.010945,50.907644,-0.148570,-0.371428,14.159756,11.867936,9.576115,13.332293,...,11.239621,10.607982,14.161687,14.793326,15.622720,16.452115,10.553520,14.057711,10.421783,2024-01-01
4,11.846248,12.542436,9.043828,46.022222,-0.206702,-0.338483,14.153874,11.846248,9.538623,12.951777,...,10.419852,9.421689,13.487377,14.485540,15.021139,15.556739,10.570990,13.120987,10.634312,2024-02-01
5,11.834369,12.590495,9.078083,47.824461,-0.216643,-0.314115,14.143350,11.834369,9.525388,11.878925,...,8.119347,6.653904,12.707616,14.173060,15.001751,15.830443,10.581132,12.251171,10.834089,2024-03-01
6,11.748791,12.644937,9.111041,46.492839,-0.248447,-0.300981,13.995901,11.748791,9.501682,11.533898,...,10.511984,10.213036,11.957916,12.256864,12.680882,13.104900,10.587855,11.808596,10.942073,2024-04-01
7,11.602091,12.691814,9.138409,43.288664,-0.335908,-0.307966,13.765450,11.602091,9.438733,11.498036,...,9.488017,8.760989,12.053998,12.781026,13.336989,13.892952,10.590207,11.004548,13.963931,2024-05-01
8,11.429832,12.725457,9.162837,42.426901,-0.418495,-0.330072,13.422220,11.429832,9.437443,10.846988,...,9.039156,8.323365,11.223238,11.939029,12.315279,12.691528,10.587739,10.717240,13.963931,2024-06-01
9,11.395206,12.742653,9.189682,44.485263,-0.447488,-0.353555,13.416307,11.395206,9.374105,10.322987,...,9.241676,8.867965,10.656876,11.030587,11.364476,11.698365,10.588972,9.971242,13.804938,2024-07-01


In [14]:
acao = yf.Ticker('WIZC3.SA')
info = acao.info

In [19]:
acao.history(period="10Y", interval="1d")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2015-06-05 00:00:00-03:00,7.038382,7.323022,6.986629,7.193640,15536400,0.0,0.0
2015-06-08 00:00:00-03:00,7.302319,7.338546,7.064256,7.131535,1648300,0.0,0.0
2015-06-09 00:00:00-03:00,7.131535,7.162587,6.996978,7.064256,2161400,0.0,0.0
2015-06-10 00:00:00-03:00,7.090133,7.281618,7.069432,7.245391,676000,0.0,0.0
2015-06-11 00:00:00-03:00,7.250566,7.504155,7.167762,7.379948,1118600,0.0,0.0
...,...,...,...,...,...,...,...
2025-02-24 00:00:00-03:00,5.930000,5.980000,5.840000,5.870000,436800,0.0,0.0
2025-02-25 00:00:00-03:00,5.850000,6.130000,5.850000,6.070000,548000,0.0,0.0
2025-02-26 00:00:00-03:00,6.020000,6.070000,5.930000,5.940000,408700,0.0,0.0


# Valuation

In [2]:
ticker = 'WHRL3'
query = f"Qual é o valuation da empresa {ticker}"
chat_valu = ModeloValuation(query=f"Qual é o valuation da empresa {ticker}", ticker=ticker, anos_projecao=5, taxa_crescimento_perpetuidade=0.014, calculo_necessidade_capital_de_giro=False, stream=True)

response= chat_valu.chat_valuation()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Diferença entre valuation e cotação: -59.25%


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
for chunk in response: 
    print(chunk, end="", flush=True)

<think>
Okay, so I need to figure out if WHRL3 is undervalued or overvalued. I have two valuation methods here: Gordon Growth Model and Discounted Cash Flow (DCF). The current stock price is 3.88, while the Gordon model gives a valuation of 1.58 and the DCF is negative at -1.9. Hmm, that's a big difference. 

First, I should compare the two valuation methods. The Gordon model is based on dividends, assuming perpetual growth, while DCF looks at future cash flows. The Gordon valuation is 1.58, which is way below the current price of 3.88. The DCF is even lower, negative, which is strange. Usually, DCF should be positive if the company is generating cash. Maybe the model is expecting negative cash flows in the future.

Looking at the DCF inputs, the revenue is increasing each year, from 11.48 billion in 2025 to 12.49 billion in 2029. EBIT is also rising, so the company seems profitable. Tax expenses are going up too, which is normal. CAPEX is increasing each year, but depreciation is as w

# Chat retirando ticker

In [ ]:
import json
with open('dados/empresas.json', 'r') as f:
    dados_empresa = json.load(f)

In [88]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from chat_bots import get_secret_key
from pydantic import SecretStr
from langchain_core.prompts import PromptTemplate
from typing import Dict

load_dotenv()

try:
    api_secret_groq = get_secret_key("GROQ_API_KEY")
except KeyError as exc:
    raise ValueError("API key inválida ou não definida") from exc

load_dotenv()


def ChatPegaTicker(
    query: str,
    dados_empresas:Dict[str,str],
    api_secret: SecretStr | None = api_secret_groq,
    modelo_llm: str = "llama-3.2-3b-preview",
) -> str:
    model = ChatGroq(
        api_key=api_secret,
        model=modelo_llm,
        temperature=0.5,
        stop_sequences=None,
    )

    prompt = PromptTemplate(
        input_variables=["query", "dados_empresas"],
        template="""
        
                Você recebe a seguinte consulta: "{query}"
                
                Sua única missão é identificar o ticker da empresa mencionada, caso exista.

                Extraia o nome da empresa na consulta.
                Encontre seu código de negociação na bolsa de valores.
                Responda apenas com o ticker, sem qualquer outra informação adicional.
                Exemplo:
                Entrada: "O que você pode me dizer sobre o banco Bradesco?"
                Saída: BBDC4
                
                Voce recebe os nomes e o codigo na empresa no arquivo {dados_empresas}
                
                Utlize esse arquivo para identificar qual e o codigo da acao
                
            De como resposta apenas o codigo da acao, sem nenhum comentario.

            Se não encontrar um ticker válido, responda que nao encontrou.

      """,
    )

    llm_chain = prompt | model | StrOutputParser()

    resposta = llm_chain.invoke({'query':query, 'dados_empresas':dados_empresas})

    return resposta

In [90]:
query = "O que vc pode me dizer da petrobras"
response = ChatPegaTicker(query=query, dados_empresas=dados_empresa)
response

'PETR3'

In [1]:
from juncao_modelos_dados import ModeloFundamentos, ModeloValuation, ModeloSentimento, ModeloAnaliseTecnica

USER_AGENT environment variable not set, consider setting it to identify your requests.


# Pegando icone da empresa

In [17]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

class PegandoLogotipo:
    def __init__(self, ticker: str) -> None:
        self.ticker = ticker
        
    def options(self) -> webdriver.ChromeOptions:
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--disable-features=NetworkService")
        chrome_options.add_argument("--window-size=1920x1080")
        chrome_options.add_argument("--disable-features=VizDisplayCompositor")
        return chrome_options

    def driver(self) -> webdriver.Chrome:
        driver = webdriver.Chrome(options=self.options())
        return driver

    def pegar_logotipo(self) -> str | None:
        driver = self.driver()
        driver.get(f"https://br.tradingview.com/symbols/BMFBOVESPA-{self.ticker}/")
        image_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[4]/div[2]/div[1]/div[1]/div/div/div/div[1]/img[3]"))
            )
            
            # Extrai o link da imagem (atributo 'src')
        image_link = image_element.get_attribute("src")
        return image_link




In [19]:
ticker = 'PETR3'
pegar_logotipo = PegandoLogotipo(ticker=ticker)
pegar_logotipo.pegar_logotipo()

'https://s3-symbol-logo.tradingview.com/brasileiro-petrobras--big.svg'

# f

In [8]:
import yfinance as yf
ticker = 'VALE3.SA'
acao = yf.Ticker(ticker)
info = acao.info
response = ChatTradutor(info['longBusinessSummary'])
response.split('\n\n')[-1]

'A Vale S.A. é uma empresa que produz e vende minério de ferro, minério de ferro em grânulos, níquel e cobre no Brasil e internacionalmente. A empresa opera em dois segmentos: Soluções de Ferro e Materiais de Transição Energética. O primeiro segmento produz e extrai minério de ferro e grânulos, além de outros produtos ferrosos, e fornece serviços logísticos relacionados. O segundo segmento produz e extrai níquel e seus subprodutos, como ouro, prata, cobalto e metais preciosos, além de cobre utilizado na construção de tubulações e fios elétricos.'

In [1]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader


In [1]:
from tratando_dados import TratarDadosNoticias
from coleta_dados import LinksExtractorBS4
from selenium import webdriver
import os
from pydantic import SecretStr

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
api_secret_groq = os.getenv("GROQ_API_KEY")
api_secret_serper = os.getenv("API_KEY_SERPER")


In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-features=NetworkService")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-features=VizDisplayCompositor")

In [4]:
tratando_dados = TratarDadosNoticias(acao='PETR3', options=chrome_options, api_secret_groq=SecretStr(api_secret_groq), api_secret_serper=SecretStr(api_secret_serper))

noticias = tratando_dados.clean_chat_html_bs4()

Error fetching URL: Redirect response '302 Found' for url 'https://br.advfn.com/noticias/PAPERBR/2025/artigo/95596556'
Redirect location: '/jornal/2025/03/petrobras-celebra-acordo-de-us-283-milhoes-para-encerrar-litigio-com-a-eig-energy-nos-eua'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/302


In [5]:
noticias.split('\nNew notice\n')

['',
 'mercado financeiro\nIbovespa hoje: Embraer (EMBR3) salta mais de 8% e Petrobras (PETR3) tem 7ª queda seguida\nÍndice fechou em alta em sessão mais curta, com mercado de olho em alívio nas tarifas de Trump\nO Ibovespa é o principal índice da B3, a Bolsa de Valores brasileira. Foto: Divulgação/B3\nO Ibovespa hoje fechou em alta, em dia de pregão reduzido, na volta do feriado de carnaval. Investidores monitoraram sinais de alívio nas tarifas aplicadas por Donald Trump contra México e Canadá, enquanto reagiram a novos dados de emprego nos Estados Unidos. A principal referência da B3 encerrou esta quarta-feira (5) em valorização de 0,20% aos 123.046,85 pontos, depois de oscilar entre máxima a 123.364,03 pontos e mínima a 122.747,06 pontos.\nPagamento',
 ' Luiza (MGLU3) anuncia unificação de vice-presidências da companhia\nDay Trade: IRB (IRBR3) e outras ações para vender hoje (10) e buscar retornos de até 3,01%\nDay Trade: BRF (BRFS3), Ambev (ABEV3) e mais 1 ação para comprar nesta s

In [8]:
text_bs4 = LinksExtractorBS4()
text = text_bs4.clean_text_bs4(url='https://br.advfn.com/noticias/PAPERBR/2025/artigo/95596556')

Error fetching URL: Redirect response '302 Found' for url 'https://br.advfn.com/noticias/PAPERBR/2025/artigo/95596556'
Redirect location: '/jornal/2025/03/petrobras-celebra-acordo-de-us-283-milhoes-para-encerrar-litigio-com-a-eig-energy-nos-eua'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/302


In [18]:
def test_dados_fundamentalistas_completo():
    df = DadosFundamentalistas("PETR4").dados_fundamentalistas_completo()
    assert isinstance(df, pd.DataFrame)
    assert not df.empty

In [23]:
df = DadosFundamentalistas("PETR4")

In [24]:
df.dados_fundamentalistas_completo()

,receita_liquida,resultado_bruto,ebit,amortizacao,ebitda,lucro_liquido,lucro_por_acao,datas,tic,capex_tres_meses,...,retorno_sobre_patrimonio_liquido_inicial,retorno_sobre_ativo_inicial,margem_bruta,margem_liquida,margem_ebit,margem_ebitda,giro_do_ativo_inicial,alavancagem_financeira,passivo_patrimonio_liquido,divida_liquida_ebitda
0,129580.0,66580.0,46460.0,-16540.0,63000.0,32560.0,2.50,2024-09-30,PETR4,-24640.0,...,0.219,0.083,0.518,0.168,0.325,0.478,0.49,2.74,1.74,1.14
1,122260.0,61050.0,33540.0,-16360.0,49900.0,-2610.0,- 0.20,2024-06-30,PETR4,-15260.0,...,0.212,0.080,0.522,0.158,0.330,0.486,0.50,2.83,1.83,1.19
2,117720.0,60700.0,44030.0,-16650.0,60680.0,23700.0,1.82,2024-03-31,PETR4,-14050.0,...,0.274,0.113,0.525,0.224,0.353,0.511,0.50,2.62,1.62,1.00
3,134260.0,72630.0,39630.0,-20490.0,60120.0,31040.0,2.38,2023-12-31,PETR4,-17790.0,...,0.344,0.128,0.527,0.243,0.370,0.518,0.52,2.76,1.76,0.91
4,124830.0,66320.0,47550.0,-19480.0,67030.0,26630.0,2.04,2023-09-30,PETR4,-15540.0,...,0.368,0.144,0.511,0.255,0.387,0.527,0.57,2.66,1.66,0.87
5,113840.0,57680.0,41970.0,-18280.0,60250.0,28780.0,2.21,2023-06-30,PETR4,-14390.0,...,0.381,0.156,0.506,0.269,0.403,0.531,0.58,2.67,1.67,0.74
6,139070.0,73310.0,60200.0,-17770.0,77970.0,38160.0,2.93,2023-03-31,PETR4,-12590.0,...,0.419,0.182,0.521,0.285,0.453,0.569,0.64,2.43,1.43,0.60
7,158580.0,76640.0,57810.0,-19460.0,77270.0,43340.0,3.32,2022-12-31,PETR4,-18710.0,...,0.486,0.194,0.521,0.294,0.459,0.576,0.66,2.69,1.69,0.65
8,170080.0,86840.0,74610.0,-18540.0,93150.0,46100.0,3.53,2022-09-30,PETR4,-10230.0,...,0.478,0.181,0.513,0.286,0.478,0.598,0.63,2.54,1.54,0.73
9,170960.0,95860.0,96430.0,-18620.0,115050.0,54330.0,4.16,2022-06-30,PETR4,-8380.0,...,0.454,0.172,0.509,0.284,0.504,0.634,0.60,2.45,1.45,0.54


In [19]:
test_dados_fundamentalistas_completo()

#